<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [1]:
!pip install pycuda

#3. Desarrollo

##3.1 Desarrollo en CPU

In [2]:
#---------------------PARAMETROS-------------------------------
#@title 3.1.1 Parametros de ejecución {vertical-output: true}

cantidadElementos = 1000000#@param {type: "number"}

import numpy
import time

tiempoTotal = time.time() * 1000

vector = numpy.random.randint(1000, size = cantidadElementos)

#----------------------ALGORITMO----------------------------
tiempoBucle = time.time() *1000

#----------------------FINALIZACION----------------------------
tiempoBucle = (time.time() *1000) - tiempoBucle

tiempoTotal = (time.time() *1000) - tiempoTotal
print("\n")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")



Tiempo del bucle:  0.0263671875 [ms.]
Tiempo del total:  9.9521484375 [ms.]


##3.2 Desarrollo en GPU

In [20]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  20#@param {type: "number"}
orden_taylor = 6#@param {type: "number"}
punto_evalacion =  3#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import time
import numpy

tiempoTotal = (time.time() * 1000)

# Vectores en CPU.
coeficientes_cpu = numpy.random.randint(10, size = orden_polinomio + 1 )
#coeficientes_cpu = numpy.array([3,2,3,4])
coeficientes_cpu = coeficientes_cpu.astype(numpy.int32())

print("Coeficientes: ")
print(coeficientes_cpu)

terminos_taylor_cpu = numpy.zeros(orden_taylor + 1)
terminos_taylor_cpu = terminos_taylor_cpu.astype(numpy.int32())

resultado_cpu = numpy.zeros(orden_taylor + 1)
resultado_cpu = resultado_cpu.astype(numpy.int32())

derivadas_cpu = numpy.arange(orden_polinomio + 2)
derivadas_cpu = derivadas_cpu.astype(numpy.int32())

print("Derivadas")
print(derivadas_cpu)
# Vectores en GPU.
coeficientes_gpu = cuda.mem_alloc(coeficientes_cpu.nbytes)
terminos_taylor_gpu = cuda.mem_alloc(terminos_taylor_cpu.nbytes)
derivadas_gpu = cuda.mem_alloc(derivadas_cpu.nbytes)

# Copio los datos
cuda.memcpy_htod(coeficientes_gpu, coeficientes_cpu)
cuda.memcpy_htod(terminos_taylor_gpu, terminos_taylor_cpu)
cuda.memcpy_htod(derivadas_gpu, derivadas_cpu)

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor( int punto, 
                               int long_pol, 
                               int *C, 
                               int *T,
                               int *D){
  
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
  int valor = 0;
  int idx_polinomio = 0;
  int aux = 0;  
  
  if(idx == 0){
    valor += C[idx_polinomio];
    for(idx_polinomio = idx + 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      aux += C[idx_polinomio] * pow((double) punto, (double) D[idx_polinomio]); 
    }
    valor += aux;
  } else 
  {
    valor += C[idx_polinomio];

    for(idx_polinomio = idx + 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      int termino = C[idx_polinomio] * D[idx_polinomio];
      C[idx_polinomio] = termino;
      D[idx_polinomio] = D[idx_polinomio] - 1;
      aux += termino * pow((double) punto, (double) D[idx_polinomio]);
    }   
  }  
  valor += aux;
  T[idx] = valor; 
}
""")
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = (time.time() * 1000)

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (orden_taylor + 1 + dim_hilo-1) / dim_hilo )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel(numpy.int32(punto_evalacion),
       numpy.int32(orden_polinomio + 1),
       coeficientes_gpu, terminos_taylor_gpu, derivadas_gpu, 
       block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1))

cuda.memcpy_dtoh(resultado_cpu, terminos_taylor_gpu)

tiempo_gpu = (time.time() * 1000) - tiempo_gpu

tiempo_total = (time.time() * 1000) - tiempoTotal
print(resultado_cpu)
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_total, "[ms]" )
print("Tiempo GPU: ", tiempo_gpu, "[ms]" )

Coeficientes: 
[7 1 4 9 4 4 7 5 1 5 8 8 1 4 0 4 8 6 7 1 7]
Derivadas
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
[ 419911 1601593 1331323 1285396 1264984 1244572  881692]
Thread x:  256 , Bloque x: 1
Thread x:  256 , Bloque x: 1
Tiempo CPU:  3.283447265625 [ms]
Tiempo GPU:  0.5224609375 [ms]


#4. Tabla de pasos

#5. Conclusiones